# Импорт функции для загрузки данных из файла  

Функция принимает:  

+ путь до файла (строка)  
+ имя таблицы (строка)  
+ столбцы для выгрузки (кортеж)  
+ столбцы которы преобразуются в числовые данные (кортеж)  
+ столбец по которому объединяются таблицы, данные в столбце должны быть числовые
+ начальная/конечная строка с данными, по умолчанию определяется автоматически (кортеж)  


In [ ]:
import string
import pandas as pd
import numpy as np
import os
def load_sheet(path_to_file: str, sheet_name: str, columns_range: tuple, numeric_columns: tuple,\
        merge_by_column: str="A", rows_range: list=None)->pd.DataFrame:
    ''' Load sheet from excel file
    ----------
    Parameters:
    path_to_file : path to excel file (str)
    sheet_name : name of sheet from excel file (str)
    columns_range : tuple with columns headers, max 702 columns (str)
    numeric_columns : tuple with columns headers, that should be numeric (str)
    merge_by_column : column by which sheets will be merged (str), by default it is first column - "A", column should be numeric
    rows_range : list with start/end numeric data row indexes (int inside list), by default define range automatically
    -------
    Returns:
    pd_sheet : pandas DataFrame
    '''
    ### Проверяем что файл существует
    if not os.path.exists(path_to_file):
        error_message = f"Не нахожу такого файла: {path_to_file}"
        raise ValueError(error_message)
    ### Формируем заголовок для конечной таблицы
    header = os.path.basename(path_to_file)
    header = header.rsplit(".", maxsplit=1)[0]
    ### Проверяем, что нет повторяющихся столбцов
    if len(set(columns_range)) != len(columns_range):
        error_message = f"Есть повторяющиеся столбцы в {header}"
        raise ValueError(error_message)
    ### Проверяем что столбцы для чисел перечислены в кортеже
    if type(numeric_columns) != tuple:
        error_message = f"Столбцы для чисел должны быть перечислены в кортежах, например так (\"A\",)"
        raise ValueError(error_message)
    ### Create list with capital letters A,B,C ... J
    alphabetcnt = 26
    columns_name = list(string.ascii_uppercase[:alphabetcnt])
    for i in range(alphabetcnt):
        for j in range(alphabetcnt):
            columns_name.append(columns_name[i]+columns_name[j])
    ### Создаем числовый список нужных столбцов
    columns = [columns_name.index(x) for x in columns_range]
    ### Загружаем файл
    num_xlsx = pd.ExcelFile(path_to_file)
    pd_sheet = pd.read_excel(num_xlsx, sheet_name, header=None, usecols=columns)
    pd_sheet.columns = columns_range
    ### Делаем реиндекс как в Excel
    pd_sheet = pd_sheet.set_index((i+1 for i in pd_sheet.index))
    ### Оставляем только нужные столбцы
    pd_sheet = pd_sheet.loc[:, columns_range]
    ### Конвертируем в числа и строки, не числа становятся NaN
    pd_sheet_num = pd.to_numeric(pd_sheet[merge_by_column], errors="coerce", downcast="integer")
    ### Определяем строки с данными в нужных столбцах по столбцу для объединений из numeric_columns
    rows_range = rows_range or [None, None]
    if rows_range[0] is None:
        rows_range[0] = pd_sheet_num.first_valid_index()
    if rows_range[0] is None:
        error_message = f"Не найдено чисел в столбце {merge_by_column} в файле {header}"
        raise ValueError(error_message)
    if rows_range[1] is None:
        rows_range[1] = pd_sheet_num.last_valid_index()
    if rows_range[1] is None:
        error_message = f"Не найдено чисел в столбце {merge_by_column} в файле {header}"
        raise ValueError(error_message)
    ### Удаляем лишние строки
    pd_sheet = pd_sheet.loc[rows_range[0]:rows_range[1],]
    ### Приводим типы к числовым еще раз, потому что иначе иногда остаются типы object
    for i in range(len(numeric_columns)):
        pd_sheet.iloc[:,i] = pd.to_numeric(pd_sheet.iloc[:,i],downcast="integer")
    ### Устанавливаем мультииндекс
    pd_sheet.columns = pd.MultiIndex.from_product([[header], pd_sheet.columns])
    print(pd_sheet.head(2))
    print("....................")
    print(pd_sheet.tail(2).to_string(header=False))
    print(" ")
    return pd_sheet

# Загружаем настройки

Важен порядок, первым должен идти файл с номенклатурой, потом файлы отчетов.  
Порядок и количество должны сохранятся во всех переменных и быть одинаковыми.


In [ ]:
### Настройки
imported_data = (("LFLАпрель/LFL_list.xlsx","TDSheet",("A", "B")),
                 ("LFLАпрель/ИМ/апрель_2021.xlsx","TDSheet",("A", "H")),
                 ("LFLАпрель/ИМ/апрель_2021.xlsx","TDSheet",("A", "I")))
                #    "LFLАпрель/ИМ/апрель_2021.xlsx",
                #    "LFLАпрель/ИМ/апрель_2021.xlsx",
                #    "LFLАпрель/ИМ/апрель_2021.xlsx",
                #    "LFLАпрель/ИМ/апрель_2021.xlsx")
# rows_range = ((6, 6835),
#               (10, 3442),
#               (10, 3442),
#               (10, 3442),
#               (10, 3442),
#               (10, 3442))

# Загружаем номенклатуру


In [ ]:
lfl = load_sheet(imported_data[0][0], imported_data[0][1], imported_data[0][2], ("A",))

# Проверяем на дубликаты в столбце кодов и удаляем их

Считаем здесь, что это первый столбец


In [ ]:
duplicated_codes = lfl[lfl.duplicated([lfl.columns[0]], keep=False)]
if not duplicated_codes.empty:
    print("Дубликаты")
    print(duplicated_codes)
    lfl.drop_duplicates(lfl.columns[0], inplace=True)

# Загружаем наши файлы с отчетами


In [ ]:
reports = []
for i in range(1, len(imported_data)):
    reports.append(load_sheet(imported_data[i][0], imported_data[i][1], imported_data[i][2], imported_data[i][2]))

# Проверяем на дубликаты по кодам и объединяем одинаковые

Считаем здесь, что столбец с кодами это первый столбец


In [ ]:
for report in reports:
    duplicated_codes = report[report.duplicated([report.columns[0]], keep=False)]
    if not duplicated_codes.empty:
        print("Дубликаты в таблице", report.columns[0][0])
        print(duplicated_codes)
        print("..............")
        col = [report.columns[cl] for cl in range(1, len(report.columns))]
        report.loc[duplicated_codes.iloc[0].name,col] = duplicated_codes[col].sum(axis=0)
        report.drop_duplicates([report.columns[0]], inplace=True)

# Присоединить по номенклатуре (ВПР)


In [ ]:
result = pd.merge(lfl, reports[0], how="left", left_on=[lfl.columns[0]], right_on=[reports[0].columns[0]])
result.drop(columns=reports[0].columns[0], inplace=True)
del reports[0]
for i,report in enumerate(reports):
    result = pd.merge(result, report, how="left", left_on=[result.columns[0]], right_on=[report.columns[0]])
    try:
        result.drop(columns=report.columns[0], inplace=True)
    except KeyError:
        result.drop(columns=(f"{report.columns[0][0]}_y", report.columns[0][1]), inplace=True)

    if i%2 == 0:
        r1 = result.iloc[:,-2]
        r2 = result.iloc[:,-1]
        r2.replace(0, np.NaN, inplace=True)
        result.loc[(r1.notna())&(r2.isna()), ("Прирост, %", report.columns[0][0])] = 999
        result["Прирост, %", report.columns[0][0]] = (r1-r2)/r2*100
        result["Прирост, %", report.columns[0][0]] = result["Прирост, %", report.columns[0][0]].apply("{:.2f}%".format)
print(result)

# Скопировать результат в буфер обмена

In [ ]:
result.to_clipboard(excel=True, index=False, float_format="%.2f", decimal=",")